In [19]:
import os

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import CSVLoader
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [3]:
from dotenv import load_dotenv

load_dotenv()
os.getenv("LANGCHAIN_PROJECT")

'encore-ds36-turtorial'

### tv_repair_product_information

In [6]:
# TV_repair.csv 불러오기
loader = CSVLoader('./TV_repair.csv', encoding='utf-8')
tv_repair_documents = loader.load()

In [7]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [8]:
# TV_repair_PINECONE vector_db 
TV_PINECONE_INDEX_NAME = os.getenv('TV_PINECONE_INDEX_NAME')
TV_PINECONE_NAMESPACE = os.getenv('TV_PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

def tv_repair_product_information(tv_repair_query):
    """
    :param tv_repair_query: 전자제품 모델명 + 수리품목 텍스트 
    :return: 
    """
    
    # 클라이언트 객체
    tv_repair_vector_db = PineconeVectorStore.from_documents(
        tv_repair_documents,
        embedding=embeddings,
        index_name=TV_PINECONE_INDEX_NAME,
        namespace=TV_PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    # 유사도 검색
    results = tv_repair_vector_db.similarity_search(
    tv_repair_query,
    k=3,
    namespace=TV_PINECONE_NAMESPACE
    )
    return {
        'as_details' : tv_repair_query,
        'repair_information' : '\n'.join([doc.page_content for doc in results])
    }

In [10]:
tv_repair_product_information('NEO QLED 4K TV 43인치의 디스플레이 수리비용은 얼마인가요?')

{'as_details': 'NEO QLED 4K TV 43인치의 디스플레이 수리비용은 얼마인가요?',
 'repair_information': '\ufeff모델명: KQ43QNA90AFXKR\n이름: NEO QLED 4K TV 43형\n부품: 디스플레이\n평균가: 623,000\n\ufeff모델명: KQ43QNA90AFXKR\n이름: NEO QLED 4K TV 43형\n부품: 디스플레이\n평균가: 623,000\n\ufeff모델명: KQ43QNA91AFXKR\n이름: NEO QLED 4K TV 44형\n부품: 케이블\n평균가: 10,500'}

In [15]:
tv_repair_product_chain = RunnableLambda(tv_repair_product_information)
input_data = {'text': '프레임 TV 55인치 리모컨 망가졌어요'}
tv_repair_product_chain.invoke(input_data['text'])

{'as_details': '프레임 TV 55인치 리모컨 망가졌어요',
 'repair_information': '\ufeff모델명: KQ55LSA05AFXKR\n이름: 프레임 TV 57형\n부품: 리모컨\n평균가: 45,000\n\ufeff모델명: KQ55LSA05AFXKR\n이름: 프레임 TV 57형\n부품: 리모컨\n평균가: 45,000\n\ufeff모델명: KQ55LSA05AFXKR\n이름: 프레임 TV 57형\n부품: 리모컨\n평균가: 45,000'}

### tv_repair_price

In [16]:

def tv_product_repair_cost(tv_repair_query):
    
    prompt = ChatPromptTemplate.from_messages([
        ('system', """
Persona:
You are a service center employee with a deep love and passion for electronics. You have extensive knowledge about electronic products and are well-versed in the necessary components and repair costs associated with different models and issues. You ensure that customers understand the repair process and the costs involved, and your tone is friendly, approachable, and easy to understand, even for beginners.

Role:
As a service center employee, your role is to explain the necessary repair parts in an easy-to-understand way and provide the repair cost information. You help customers understand that only the appropriate minimum repair parts will be used, ensuring they have a seamless experience. Your goal is to improve the customer's inconvenience by recommending the right parts and repair costs so they can continue to use their electronics conveniently.

Examples:
**Example of screen damage repair**:
If a customer requests an explanation for the repair cost due to screen damage, you would provide information on the product's model, the warranty period, the appropriate repair parts, the cost, and the estimated repair time, explaining everything in a way that even a child can understand.
        """),
        HumanMessagePromptTemplate.from_template('''
        수리비용 설명에 아래의 제품명과 모델명, 수리부품, 수리비용만을 참고하여 한글로 답변해주세요.
        A/S 내용 :
        {as_details}
        수리 정보 :
        {repair_information}
        ''')
    ])
    
    model = ChatOpenAI(
        model = 'gpt-4o',
        temperature = 0.3
    )
    
    output_parser = StrOutputParser()
    
    return prompt | model | output_parser
    

In [17]:
chain = RunnableLambda(tv_repair_product_information) | RunnableLambda(tv_product_repair_cost)

input_data = {'text': 'NEO QLED 4K TV 45인치 디스플레이 수리 비용은 얼마인가요?'}
print(chain.invoke(input_data['text']))

안녕하세요! NEO QLED 4K TV 43형의 디스플레이 수리에 대해 문의 주셨군요. 해당 모델명은 KQ43QNA90AFXKR입니다. 디스플레이 수리에 필요한 부품의 평균 비용은 623,000원입니다. 

수리 과정은 디스플레이 교체가 주로 이루어지며, 수리 시간은 부품의 재고 상황에 따라 달라질 수 있습니다. 수리비용에는 부품비와 작업비가 포함될 수 있으니, 정확한 견적을 위해서는 서비스 센터 방문을 추천드립니다. 

추가로 궁금한 점이 있거나 도움이 필요하시면 언제든지 문의해 주세요! 감사합니다.


In [18]:
input_data = {'text': '프레임 TV 88인치 리모컨 잃어버렸는데 교체 비용 알려주세요'}
print(chain.invoke(input_data['text']))

안녕하세요! 프레임 TV 87형의 리모컨 교체 비용에 대해 안내드리겠습니다. 고객님께서 사용 중이신 모델명은 KQ85LSA05AFXKR이며, 이 모델의 리모컨 평균 교체 비용은 45,000원입니다. 리모컨은 TV를 편리하게 조작할 수 있도록 도와주는 중요한 부품이니, 필요하신 경우 언제든지 교체를 진행하실 수 있습니다. 추가적인 문의사항이 있으시면 언제든지 말씀해 주세요!
